# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [2]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BarnesDoubleGamma, BenchmarkTools
println("Number of threads: ", Threads.nthreads())

Number of threads: 64


# Bootstrap equations

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}
We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [3]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
P = big"0.2" + big"0.1" * im
diag_field = Field(c, P=P)
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(
    indices, fields=nothing;
    c=CC(β=1 / (big"0.8" + big"0.2" * im)),
    P=big"0.2" + big"0.1" * im,
    parity, precision=10
)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    if fields === nothing
        ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
        fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))
    end
    co = Correlation([Field(c, r=r, s=s) for (r, s) in indices], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

function compute_diagblocks(specs, Ps)
    co = specs.s.corr
    c = co.fields[1].c
    return @channels [IBlock(co, chan, Field(c, P=P)) for P in Ps[chan]]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
    specs, signature;
    even_spin=Channels(false),
    rmax=3, show=true, Ps=nothing,
    fix=nothing,
    rels=nothing,
)
    specs = @channels filter(V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels)
        if show
            if rels === :stu
                printstyled("Channels s, t, u:\n", bold=true)
                Base.show(stdout, res.str_cst.s, rmax=rmax)
            else
                Base.show(stdout, res.str_cst, rmax=rmax)
            end
        end
    else
        diagblocks = compute_diagblocks(specs, Ps)
        res = solve_bootstrap(specs, diagblocks, fix=fix, rels=rels)
    end
    return res
end;

# Some numerical four-point functions

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [8]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, fields, parity=0, precision=20);

### Signature $(0, 1, 1)$

In [9]:
sol = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ P = (0.2+0.1im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ -0.167173+0.134319im      │ 5e-18     │
│ (2, 0)          │ -0.00239852+0.000773346im │ 2.4e-16   │
│ (2, 1)          │ 0.000797319-0.000705013im │ 1.4e-16   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌────────────┬───────────────────────────┬───────────┐
│ Field      │ Structure constant        │ Rel. err. │
├────────────┼───────────────────────────┼───────────┤
│ (1, 0)     │ 0.422102+0.0893706im      │ 1.8e-18   │
│ (1, 1)     │ 0.16568-0.0434621im       │ 8.9e-19   │
│ (2, 0)     │ 0.000999679-1.2823e-05im  │ 4.6e-16   │
│ (2, -1//2) │ 0.000592214+0.000185033im │ 1.1e-15   │
│ (2, 1//2)  │ 0.000592214+0.000185033im │ 9.7e-16   │
│ (2, 1)     │ 0.000344058+0.000230614im │ 6.7e-16   │
└──

In [6]:
# latex output renders in the notebook
display(c)
display(sol.correlation)
# latex copy-pastable output with backend=:latex keyword argument:
show(stdout, sol.str_cst, rmax=2, backend=:latex)

c = 1.614532871972318339100346020761245674740484429065743944636678200692041522491287 + 2.232249134948096885813148788927335640138408304498269896193771626297577854671372im, β = -1.176470588235294117647058823529411764705882352941176470588235294117647058823537 + 0.2941176470588235294117647058823529411764705882352941176470588235294117647058841im

NonChiralCorrelation{Complex{BigFloat}} with external fields
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >

 Channel s
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Rel. err.} \\\hline
  $P = (0.2+0.1im)$ & $1.0$ & 0 \\
  $\left(1,0\right)$ & $-0.16717252\;+\;0.13431905\mathrm{i}$ & 4.2e-18 \\
  $\left(2,0\right)$ & $-0.002398521\;+\;0.000773346\mathrm{i}$ & 1.8e-16 \\
  $\left(2,1\right)$ & $0.0007973192\;-\;0.00070501294\mathrm{i}$ & 1.2e-16 \\\hline
\end{tabular}
\\ Channel t
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Rel. err.} \\\hline
  $\left(1,0\right)$ & $0.42210183\;+\;0.08937056\mathrm{i}$ & 1.6e-18 \\
  $\left(1,1\right)$ & $0.16567983\;-\;0.04346215\mathrm{i}$ & 6.9e-19 \\
  $\left(2,0\right)$ & $0.0009996791\;-\;1.2822965e-5\mathrm{i}$ & 3.9e-16 \\
  $\left(2,\frac{-1}{2}\right)$ & $0.0005922143\;+\;0.00018503261\mathrm{i}$ & 8.3e-16 \\
  $\left(2,\frac{1}{2}\right)$ & $0.0005922143\;+\;0.00018503261\mathrm{i}$ & 9e-16 \\
  $\left(2,1\right)$ & $0.00034405786\;+\;0.00023061357\mathrm{i}$ & 6e-1

### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [5]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16517-0.591022im │    7.65818e-15 │
│          (1, 1) │     0.464566+0.0798147im │     8.4386e-15 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│       (2, 1//2) │ -0.00318409+0.00145081im │    1.01034e-14 │
│      (2, -1//2) │ -0.00318409+0.00145081im │     8.2106e-15 │
│          (2, 1) │  0.00257332-0.00274978im │    6.86828e-15 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433576-0.577531im │    1.38597e-14 │
│     (1, 1) │     0.672446+0.0319329i

### Solve for many different Ps

In [78]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s)
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s)
# print one of the results
show(stdout, sol_Ps[2].str_cst, rmax=2)

InterchiralBlock{Complex{BigFloat}}[G^(s)({ V_{P = 0.80000000000000000000000017 + 0.21000000000000000000000003im + n/β}, n ∈ -5:1:3}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.22000000000000000000000002im + n/β}, n ∈ -5:1:4}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.23000000000000000000000002im + n/β}, n ∈ -5:1:4})]
 Channel s
┌─────────────────┬─────────────────────────┬───────────┐
│ Field           │ Structure constant      │ Rel. err. │
├─────────────────┼─────────────────────────┼───────────┤
│ (1, 0)          │ 0.253897-0.354829im     │ 5.3e-18   │
│ (P=0.80+0.22im) │ 1.0+0.0im               │ 0         │
│ (2, 0)          │ 0.00488132-0.00418444im │ 2.1e-16   │
│ (2, 1)          │ -0.00118272+0.0024477im │ 1.2e-16   │
└─────────────────┴─────────────────────────┴───────────┘
 Channel t
┌────────────┬─────────────────────────┬───────────┐
│ Field      │ Structure constant      │ Rel. err. │
├────────────┼─────────────────────────┼───────────┤
│ (1, 0)     │ -1.0826

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\left\langle P_1 P_2 P_3 P_4 \right\rangle$

In [7]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields, parity=1, precision=15);

In [8]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.043652+0.0461863im │    6.68646e-10 │
│          (1, 1) │     -0.00502903+0.098547im │     1.1386e-10 │
│          (2, 0) │   0.00079105-0.000735744im │     4.28782e-8 │
│       (2, 1//2) │  0.000110591-0.000183974im │     1.46143e-7 │
│          (2, 1) │ -0.000189192-3.69278e-06im │     5.65625e-8 │
│          (3, 0) │                          0 │      0.0607838 │
│       (3, 1//3) │                          0 │      0.0644803 │
│       (3, 2//3) │                          0 │      0.0288652 │
│          (3, 1) │                          0 │      0.0139439 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [9]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields2, parity=1, precision=13);

UndefVarError: UndefVarError: `fields2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Signature $(0, \frac32, \frac32)$

In [10]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │              1.0+0.0im │            0.0 │
│          (1, 0) │    -1.79924+0.421555im │      0.0536187 │
│          (1, 1) │ 0.000273845+0.203333im │       0.179268 │
│          (2, 0) │      1.05837-2.37073im │       0.276612 │
│       (2, 1//2) │   -0.557097+0.386062im │        1.79628 │
│          (2, 1) │    -0.133632-0.76917im │       0.431461 │
│          (3, 0) │      41.9787-27.2899im │       0.626559 │
│       (3, 1//3) │     -34.9665+26.8837im │       0.680484 │
│       (3, 2//3) │       24.938-25.2727im │       0.667133 │
│          (3, 1) │     -11.1594+19.0543im │       0.716201 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├─

### Signature $(1, \frac12, \frac32)$

In [11]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.314676-0.227065im │        1.58253 │
│    (2, 0) │   -0.756645+5.0774im │       0.595367 │
│ (2, 1//2) │  -0.515941-3.00656im │       0.911785 │
│    (2, 1) │   0.973783+1.64767im │        1.26323 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │ -0.0288501+0.0165005im │          1.011 │
│    (1, 1) │  0.0329273-0.0770832im │       0.777737 │
│    (2, 0) │      3.39492+1.71516im │          2.017 │
│ (2, 1//2) │     -2.78334-1.07935im │        1.75393 │
│    (2, 1) │    1.69856+0.0715347im │        1.71933 │
└───────────┴────────────────────────┴──────

### Signature $(1, \frac32, \frac12)$

In [12]:
solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.422351-0.268128im │       0.215619 │
│    (2, 0) │    -2.42269+4.0059im │       0.383119 │
│ (2, 1//2) │    1.60603-2.13824im │       0.575963 │
│    (2, 1) │  -0.166614+1.56393im │       0.488147 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────┬────────────────┐
│ Field     │ Structure constant │ Relative error │
├───────────┼────────────────────┼────────────────┤
│    (2, 0) │  2.02455+1.94047im │       0.673465 │
│ (2, 1//2) │ -1.66763-1.39314im │       0.676533 │
│    (2, 1) │ 1.12007+0.374722im │       0.710549 │
└───────────┴────────────────────┴────────────────┘
 Channel u
┌───────────┬───────────────────────┬────────────────┐
│ Field     │ Structure constant    │ Relative error │
├──────

## $\left\langle (0, \frac12)^3 P_{(1, 0)} \right\rangle$

In [3]:
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
sol_diag = solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 1.35314+0.621346im        │ 3.2e-18   │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 1.9e-16   │
│ (2, 1)          │ -9.09864e-05+0.00247326im │ 1.3e-16   │
│ (3, 0)          │ 1.58764e-06+1.66872e-06im │ 1.9e-12   │
│ (3, 2//3)       │ 1.18639e-06-4.9787e-07im  │ 1.8e-12   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0-8.82188e-20im         │ 3.4e-19   │
│ (1, 0)          │ 1.35314+0.621346im        │ 3e-18     │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 9.6e-17   │
│ (2, 1)          

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [4]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);

In [5]:
sol1 = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),show=true, rmax=2);
sol2 = solve(blocks, Sig(1, 0, 1), even_spin=Channels(false, true, false), show=false);
sol3 = solve(blocks, Sig(1, 1, 0), even_spin=Channels(false, false, true), show=false);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 0                         │ 0.31      │
│ (2, 0)          │ -0.00585115-0.000436412im │ 1.5e-17   │
│ (2, 1)          │ 0.00227685-0.000981606im  │ 1.5e-17   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌───────────┬──────────────────────────┬───────────┐
│ Field     │ Structure constant       │ Rel. err. │
├───────────┼──────────────────────────┼───────────┤
│ (1, 0)    │ 0.676571+0.310673im      │ 8.6e-20   │
│ (1, 1)    │ 0.286591-0.0446828im     │ 4.8e-20   │
│ (2, 0)    │ -0.00437577+0.00233119im │ 1e-17     │
│ (2, 1//2) │ -0.00327768+0.00228679im │ 9.9e-18   │
│ (2, 1)    │ -0.00118392+0.00172743im │ 8.3e-18   │
└───────────┴──────────────────────────┴───────────┘
 Channel u
┌─────────

In [6]:
show(
    stdout,
    sol1.str_cst.s + sol2.str_cst.s + sol3.str_cst.s - sol_diag.str_cst.s,
    rmax = 2
)

┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0                          │ 0.31      │
│ (1, 1)          │ 5.17633e-20-1.52054e-19im  │ 6e-19     │
│ (2, 0)          │ -2.60874e-19-4.11794e-18im │ 4.3e-16   │
│ (2, 1//2)       │ 1.18137e-18+1.02885e-18im  │ 2.7e-16   │
│ (2, 1)          │ -6.71868e-19-5.58104e-19im │ 4e-16     │
└─────────────────┴────────────────────────────┴───────────┘


## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [16]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.98082e-17 │
│    (2, 0) │                          0 │       0.880898 │
│ (2, 1//2) │                          0 │       0.970237 │
│    (2, 1) │                          0 │        1.06903 │
│    (3, 0) │ -7.86521e-05-0.000322813im │    9.03894e-12 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │    1.86741e-11 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │    1.78644e-11 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │    9.44656e-12 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

# Polynomials

In [37]:
import BootstrapVirasoro.LoopModels: compute_reference, ρ_residue

wrs(r, s, β) = 2cospi(r * β^2 - s)

βs = [big"0.72321" + (i-1)//100 + 0im for i in 1:9] # range of βs such that ns are between 0 and 1.
cs = [CC(β=β) for β in βs]
ns = [-2cospi(β^2) for β in βs]
DGs = [DoubleGamma(c.β) for c in cs]
inds = [((0, 1 // 2), (0, β^2), (0, 1 // 2), (0, 1 // 2)) for β in βs]
Ps = [[(big"0.2" + big"0.2" * im + i // 100 * im) * cs[1].β / cs[j].β for i in 1:5] for j = eachindex(cs)] # diag field identical in each channel
blocks = [[precompute_blocks(inds[j], c=c, P=P, parity=1, precision=25) for (i, P) = enumerate(Ps[j])] for (j, c) in enumerate(cs)]
sols = [[solve(blocks[j][i], Sig(0, 0, 0), even_spin=Channels(true), show=false, rels=:stu) for i = eachindex(Ps[1])] for j = eachindex(cs)];

In [38]:
function reduce_to_polyterm(r, s)
    vals = [[sols[j][i].str_cst.s.constants[Field(c, r=r, s=s)] for i = eachindex(Ps[1])] for (j, c) = enumerate(cs)]
    Threads.@threads for j = 1:length(cs)
        c = cs[j]
        DG = DGs[j]
        V = Field(c, r=r, s=s)
        n = -2cospi(c.β^2)
        Threads.@threads for i = 1:length(Ps[j])
            P = Ps[j][i]
            co = sols[j][i].correlation
            diag_field = Field(c, P=P)
            w = 2cospi(2c.β * P)
            DP = compute_reference(co, diag_field, :s, DG)
            DV = compute_reference(co, V, :s, DG)
            vals[j][i] *= DP / DV
            vals[j][i] -= ρ_residue(V, co.fields...) / (w - wrs(V.r, V.s, c.β))
        end
    end
    return [((2cospi(2c.β * Ps[j][i]), -2cospi(c.β^2)), vals[j][i]) for i = eachindex(Ps[1]) for (j, c) = enumerate(cs)]
end

function fit_polynomial(data, varnames, degs)
    if varnames === (:w, :n) || varnames === (:n, :w)
        to_fit = data
    elseif varnames === (:w,)
        to_fit = [((d[1][1],), d[2]) for d in data[1:length(cs):end]]
    elseif varnames === (:n,)
        to_fit = [((d[1][2],), d[2]) for d in data[1:length(cs)]]
    else
        error("Unsupported varnames")
    end
    fit!(Polyfit(varnames, degs), to_fit)
end

fit_polynomial (generic function with 1 method)

## $d_{(r, s)}$ for $(r, s) = (1, 0), (2, 0), (2, 1)$

In [39]:
# only run to reset the data Dict! a bit expensive to recompute.
data = Dict()

Dict{Any, Any}()

In [40]:
for (r, s) in [(1, 0), (2, 0), (2, 1)]
    data[r, s] = reduce_to_polyterm(r, s)
end;

In [41]:
degs_w = [1, 2]

for (r, s) in [(1, 0), (2, 0), (2, 1)]
    display(fit_polynomial(data[r, s], (:w,), (degs_w[r],)))
end

Polyfit((:w,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[-2.1245513221439924214407034036027e-16 - 2.9048444978345940088595892003102e-16im, 1.9999999999999984223413275740856 + 1.5549554845065967477376763076677e-16im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[7.9581860220898753288203430685176 + 2.075420876060622857492266257197e-15im, -3.9790930110449354114131871649067 - 1.2886634173266788754324086427484e-15im, -3.9790930110449346107025163929019 + 1.3772466292885839836615800979881e-16im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[7.9581860220898917132148524313084 - 1.3916802499488162043338879698888e-14im, 3.9790930110449238472257641060319 - 8.0166961838910253020208057320812e-15im, -3.9790930110449347821562182867623 + 1.8977335883630119777032600854543e-15im])

In [27]:
data[3, 0] = reduce_to_polyterm(3, 0);
data[3, 2//3]  = reduce_to_polyterm(3, 2//3);

In [28]:
for deg in 6:7
    display(fit_polynomial(data[3, 2//3], (:n,), (deg,)))
end

Polyfit((:n,), (6,), 7, [(0,), (1,), (2,), (3,), (4,), (5,), (6,)], Complex{BigFloat}[-1.209557304568470508914406786312e+11 + 6.5378026073736521740837099756719e+11im, -3.9252438612334837861081335839302e+10 + 1.6959829307451692222820761548499e+10im, -8.5371362810363977441233021635905e+08 + 6.0973731101423639574407279673236e+08im, 1.4719177036080296163707159733627e+07 + 1.7731028038150386146716560138843e+06im, -180000.9813338163806174869673356 + 146160.93421339146332237071211165im, -2822.7738583148686207208050425226 - 3331.9894075911401775418281156896im, 16.118543753869836347520549613006 - 51.46483494595595316240079208282im])

Polyfit((:n,), (7,), 8, [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,)], Complex{BigFloat}[6.4711444225251767219738364724093e+09 + 1.8690313089664139447197119004471e+11im, -1.3949262416765557142213307547718e+10 - 1.3956286997838808449108207199162e+09im, -2.8880706409094081867782256720355e+08 - 4.0225303424211924104637798131702e+08im, -7.4287974157185576537406565677323e+06 - 5.4898574206718850173738692691914e+06im, 5828.5997007503578062599466248847 + 112320.19888310971020445699874942im, -1323.6999350081238728894276594392 - 1483.6393655849744978547763800228im, 45.181149450285048547585658453059 - 19.015720942979409157092961047599im, 0.52243832238639020218817514310074 - 0.14259268498169857910028420935809im])

In [32]:
fit30 = []
for deg in 1:4
    push!(fit30, fit_polynomial(data[3, 0], (:w,), (deg,)))
    display(fit30[end])
end

Polyfit((:w,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[-3.2444506121418570136217493369932e+07 + 8.2372418407299912890108026422825e+06im, -1.5887747340141087379166966050503e+06 + 1.323713436912829142328915393411e+07im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[1.5924097617686016986334946068614e+07 - 3.9234090254523202768005917172498e+06im, 2.1152997576260688304465893984204e+06 - 1.3331777606535826525484167068816e+07im, -3.0525285636591183614378534057719e+06 - 1.7880262044351491623673701691949e+06im])

Polyfit((:w,), (3,), 4, [(0,), (1,), (2,), (3,)], Complex{BigFloat}[-1.2051448960763617457727505475061e-13 + 6.7842557087171568548851401225445e-14im, 234401.78433616130991711576459309 - 197021.08017065537264475754057855im, 5.1799924884813940362650336277815e-14 + 1.5110545571005897392726767174212e-14im, 247591.13791360883417696395156354 - 188042.36133749951500558123958233im])

Polyfit((:w,), (4,), 5, [(0,), (1,), (2,), (3,), (4,)], Complex{BigFloat}[-1.2490607492010123619682475113939e-14 - 6.6716166540486895540773532624891e-14im, 234401.78433616130982588943180089 - 197021.08017065537280742272794851im, -2.2621467816399950080601110126146e-14 + 2.4484747588882325904652475741387e-14im, 247591.13791360883417350368952109 - 188042.3613374995149927326493163im, 6.9731459690428215684858341673598e-16 + 5.3445189438809469754017217820821e-16im])

In [33]:
fit323 = []
for deg in 1:4
    push!(fit323, fit_polynomial(data[3, 2//3], (:w,), (deg,)))
    display(fit323[end])
end

Polyfit((:w,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[-3.1622220698645908080526116659162e+07 + 8.6799341412184170640138744327377e+06im, -1.7820854597727690882474840371767e+06 + 1.3724956287741273274036528932297e+07im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[1.569498839051474057261998968592e+07 - 5.0779322186542649959441446662637e+06im, 913423.9553661534698271123873071 - 1.2642308375139817931582976641691e+07im, -3.0786804162300226687784272270984e+06 - 1.6549348382213190868058016921188e+06im])

Polyfit((:w,), (3,), 4, [(0,), (1,), (2,), (3,)], Complex{BigFloat}[114138.79323570162811087168242826 - 627671.4920828052363742786894895im, -468246.01093690644138052739851702 + 394555.68357047958056762877453797im, -1.3044073266349722307102054274292 - 0.44694783801364445298050519447909im, 237655.56193984216212893264160085 - 194628.60407957595762610362181247im])

Polyfit((:w,), (4,), 5, [(0,), (1,), (2,), (3,), (4,)], Complex{BigFloat}[114138.79375776047104169393661055 - 627671.49178680620244695653327751im, -468246.0104292078542342157293568 + 394555.68316678707284013543058915im, -1.3044809142842707052942640768284 - 0.44719812187932729446140992080707im, 237655.56189380598854482212373282 - 194628.60408430715868773813465711im, -1.4468953403013170921943376442449e-06 + 2.6913251142633596712049079600265e-06im])

In [34]:
fit323 = []
for deg in 6:7
    push!(fit323, fit_polynomial(data[3, 2//3], (:n,), (deg,)))
    display(fit323[end])
end

Polyfit((:n,), (6,), 7, [(0,), (1,), (2,), (3,), (4,), (5,), (6,)], Complex{BigFloat}[-1.209557304568470508914406786312e+11 + 6.5378026073736521740837099756719e+11im, -3.9252438612334837861081335839302e+10 + 1.6959829307451692222820761548499e+10im, -8.5371362810363977441233021635905e+08 + 6.0973731101423639574407279673236e+08im, 1.4719177036080296163707159733627e+07 + 1.7731028038150386146716560138843e+06im, -180000.9813338163806174869673356 + 146160.93421339146332237071211165im, -2822.7738583148686207208050425226 - 3331.9894075911401775418281156896im, 16.118543753869836347520549613006 - 51.46483494595595316240079208282im])

Polyfit((:n,), (7,), 8, [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,)], Complex{BigFloat}[6.4711444225251767219738364724093e+09 + 1.8690313089664139447197119004471e+11im, -1.3949262416765557142213307547718e+10 - 1.3956286997838808449108207199162e+09im, -2.8880706409094081867782256720355e+08 - 4.0225303424211924104637798131702e+08im, -7.4287974157185576537406565677323e+06 - 5.4898574206718850173738692691914e+06im, 5828.5997007503578062599466248847 + 112320.19888310971020445699874942im, -1323.6999350081238728894276594392 - 1483.6393655849744978547763800228im, 45.181149450285048547585658453059 - 19.015720942979409157092961047599im, 0.52243832238639020218817514310074 - 0.14259268498169857910028420935809im])

In [38]:
for deg in 0:8
    display(fit_polynomial(data[3, 0], (:n,), (deg,)))
end

Polyfit((:n,), (0,), 1, [(0,)], Complex{BigFloat}[-1.7961538349780760907840647978534e+13 + 4.9301220499852423309128233369358e+12im])

Polyfit((:n,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[-1.6017912645795416400239873929115e+13 + 1.1542246500741355298350241556475e+12im, -3.2308573433397134102411691368873e+11 - 6.8703059499806008698469346826151e+11im])

Polyfit((:n,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[-1.1161433443889703042696088388666e+13 + 1.2120255671101172213055265808497e+12im, -8.0256753756734209262940184115439e+10 - 5.5169704891566493173973295753663e+11im, 7.0658915621553692246618179344077e+09 - 5.5595755990709109648099953729272e+09im])

Polyfit((:n,), (3,), 4, [(0,), (1,), (2,), (3,)], Complex{BigFloat}[-5.2432592599369133763875774129973e+12 + 3.8741559620349927862673122770203e+12im, -5.727747754980468017483226442091e+10 - 3.4363257707115298238090902334428e+11im, 6.5891643526815118268208499929396e+09 - 1.0521993292181406804440995062609e+09im, 6.9404753469910388032581145259563e+07 + 4.0931553692365046683045170885349e+07im])

Polyfit((:n,), (4,), 5, [(0,), (1,), (2,), (3,), (4,)], Complex{BigFloat}[2.8573453280289364344815482322434e+12 + 9.8780823004232824240079604343964e+09im, -1.3200223259352767079917138838443e+11 - 1.3237225281126037817149086463946e+11im, 3.7523358356803584875892862182801e+09 - 2.5087688039983338338616041970381e+08im, 2.1564846532122196632670043990377e+07 + 4.9712066570940169349275754721935e+07im, -268613.73323576178215814633890814 + 670563.93922794230387415335373687im])

Polyfit((:n,), (5,), 6, [(0,), (1,), (2,), (3,), (4,), (5,)], Complex{BigFloat}[2.6563432145923248699860632816459e+11 + 1.652319592303338293530630947907e+12im, 2.574777073613468516252650568424e+09 + 9.0300408432193528765849377880803e+10im, 1.0923138274012278516787772615692e+09 - 1.0188015858083456992767554938249e+09im, 1.0636643900312187215391363293066e+07 + 2.2561644655281957593772957616778e+07im, -390900.06179843171989818316281283 + 228406.96183665633145593554059635im, -6695.1499789525130671595613673597 - 2303.8171621190118791690966917836im])

Polyfit((:n,), (6,), 7, [(0,), (1,), (2,), (3,), (4,), (5,), (6,)], Complex{BigFloat}[-1.2093600784369005005996507389106e+11 + 6.539261036835349541208454820845e+11im, -3.9259464133537608305701010808181e+10 + 1.6966133106595897234013723571116e+10im, -8.5384914027444675627224177297755e+08 + 6.0992653230907207198618038083181e+08im, 1.4722417736265883135303275592081e+07 + 1.77243862184596462383461635247e+06im, -180067.66865254115051217489772904 + 146198.61257147719879978027768392im, -2820.5898528676947594480192395121 - 3337.6916367414914309511095962783im, 13.167246154679379221538129476526 - 55.634668234077100257828941535892im])

Polyfit((:n,), (7,), 8, [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,)], Complex{BigFloat}[6.4848432935927513087810562610711e+09 + 1.8694156592354189214375640026378e+11im, -1.3952246334159585423978912876295e+10 - 1.3949772449791497982993795152118e+09im, -2.8889476686333693511759061270163e+08 - 4.023169648440587632760742274828e+08im, -7.4307377942930520472270142033348e+06 - 5.4904926380280229951709966952149e+06im, 5798.7067925333113647897003491079 + 112337.65190014189203525378253499im, -1321.0696743002300136458861122009 - 1489.0575103892381034935730087438im, 42.238264969500439592313745243999 - 23.180760317142199925719671509706im, 0.52253842276272822730922763345519 - 0.14265955074453909384948075578029im])

Polyfit((:n,), (8,), 9, [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)], Complex{BigFloat}[3.7823027410706225622509595970925e+10 + 9.9519461476320104304406310291716e+09im, -3.4418911491052972715897001993406e+09 + 6.3120073511461101188441173478451e+08im, -7.3811051494372720203355298455271e+06 - 1.530212311009419827768870381814e+08im, 2.8843169975543944163690329071084e+06 - 3.3642236945853679993230312382786e+06im, 31140.149550772070866363293060232 - 65492.421393225319984840593127709im, -924.80281101232951393830989016062 + 118.54042736832690080240430053466im, 27.094343142421030964540163249484 - 6.1107758757166690713681332211665im, 0.30451857537555969631222511981128 + 0.18310888323718351996433660364417im, 0.0019958273411659803015143707512604 + 0.0040498983479464221668245267412179im])